# 使用 LangGraph 实现对话记忆

In [ ]:
# 安装依赖
# !pip install langgraph langchain-ollama langchain-core

In [4]:
# 导入必要的库
from typing import Annotated, Dict, List, Any
from typing_extensions import TypedDict
import uuid
import json
from datetime import datetime

from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore

print("✅ 库导入完成")

✅ 库导入完成


In [2]:
# 初始化 Ollama 模型
model = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen2.5:3b",
    temperature=0.7
)

print("✅ 模型初始化完成")

✅ 模型初始化完成


## 方式1: 基础对话记忆 - 使用 MessagesState

In [5]:
## 方式1: 基础对话记忆 - 使用 MessagesState
def call_model_basic(state: MessagesState) -> dict:
    """基础模型调用"""
    system_msg = SystemMessage(content="你是一个友好的AI助手，能够记住对话历史。")
    messages = [system_msg] + state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


# 创建基础图
builder_basic = StateGraph(MessagesState)
builder_basic.add_node("call_model", call_model_basic)
builder_basic.add_edge(START, "call_model")
builder_basic.add_edge("call_model", END)

graph_basic = builder_basic.compile(checkpointer=MemorySaver())
print("✅ 方式1: 基础对话图创建完成")

✅ 方式1: 基础对话图创建完成


In [15]:
# 方式1测试：基础对话记忆
print("🧪 开始测试方式1：基础对话记忆")
print("="*60)

config1 = {"configurable": {"thread_id": str(uuid.uuid4())}}

print("\n第1轮对话:")
user_input1 = "你好，我叫张三，是一名程序员"
print(f"用户: {user_input1}")
result1 = graph_basic.invoke({"messages": [HumanMessage(content=user_input1)]}, config1)
print(f"AI: {result1['messages'][-1].content}")

print("\n第2轮对话:")
user_input2 = "你还记得我的名字和职业吗？"
print(f"用户: {user_input2}")
result2 = graph_basic.invoke({"messages": [HumanMessage(content=user_input2)]}, config1)
print(f"AI: {result2['messages'][-1].content}")

print("\n第3轮对话:")
user_input3 = "我今年30岁，住在北京"
print(f"用户: {user_input3}")
result3 = graph_basic.invoke({"messages": [HumanMessage(content=user_input3)]}, config1)
print(f"AI: {result3['messages'][-1].content}")

print("\n第4轮对话:")
user_input4 = "请总结一下我的个人信息"
print(f"用户: {user_input4}")
result4 = graph_basic.invoke({"messages": [HumanMessage(content=user_input4)]}, config1)
print(f"AI: {result4['messages'][-1].content}")

print("\n✅ 方式1测试完成")

🧪 开始测试方式1：基础对话记忆

第1轮对话:
用户: 你好，我叫张三，是一名程序员
AI: 你好，张三！很高兴认识你。作为程序员听起来很有趣也很有挑战性。你在编程方面有什么具体的项目或者问题需要帮助吗？或者你想了解一些关于编程的知识和技巧？

第2轮对话:
用户: 你还记得我的名字和职业吗？
AI: 当然记得。你叫张三，是一名程序员。如果你现在有任何问题或想聊聊编程相关的话题，我都在这里帮助你。

第3轮对话:
用户: 我今年30岁，住在北京
AI: 你好，张三！很高兴认识你。你现在30岁，并且住在北京。有什么我可以帮你了解或者解答的问题吗？或者是你想讨论某个特定的编程话题或技术问题？如果你有任何疑问或需要建议，随时告诉我！

第4轮对话:
用户: 请总结一下我的个人信息
AI: 当然可以，张三。根据你的信息：

- 年龄：30岁
- 职业：程序员
- 居住地：北京

如果有其他需要补充的信息或者想讨论的话题，请随时告知。我很乐意帮助你！

✅ 方式1测试完成


## 方式2: 带摘要的对话记忆

In [18]:
class SummaryState(TypedDict):
    messages: Annotated[List[BaseMessage], "对话消息"]
    summary: Annotated[str, "对话摘要"]


def summarize_conversation(state: SummaryState) -> dict:
    """摘要对话历史"""
    if len(state["messages"]) > 2:  # 超过10条消息时摘要
        messages_text = "\n".join([f"{msg.type}: {msg.content}" for msg in state["messages"][:-4]])
        summary_prompt = f"请简洁摘要以下对话内容：\n{messages_text}"
        summary_response = model.invoke([HumanMessage(content=summary_prompt)])
        return {
            "summary": summary_response.content,
            "messages": state["messages"][-4:]  # 保留最近4条消息
        }
    return {"summary": state.get("summary", ""), "messages": state["messages"]}


def call_model_with_summary(state: SummaryState) -> dict:
    """带摘要的模型调用"""
    system_content = "你是一个AI助手。"
    if state.get("summary"):
        system_content += f"\n对话摘要：{state['summary']}"

    system_msg = SystemMessage(content=system_content)
    messages = [system_msg] + state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


# 创建摘要图
builder_summary = StateGraph(SummaryState)
builder_summary.add_node("summarize", summarize_conversation)
builder_summary.add_node("call_model", call_model_with_summary)
builder_summary.add_edge(START, "summarize")
builder_summary.add_edge("summarize", "call_model")
builder_summary.add_edge("call_model", END)

graph_summary = builder_summary.compile(checkpointer=MemorySaver())
print("✅ 方式2: 摘要对话图创建完成")



✅ 方式2: 摘要对话图创建完成


In [19]:
# 方式2测试：带摘要的对话记忆
print("\n🧪 开始测试方式2：带摘要的对话记忆")
print("="*60)

config2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

# 初始状态
initial_state2 = {
    "messages": [],
    "summary": ""
}

print("\n第1轮对话:")
user_input1 = "你好，我叫李四，是一名数据科学家"
print(f"用户: {user_input1}")
state2 = {"messages": [HumanMessage(content=user_input1)], "summary": ""}
result1 = graph_summary.invoke(state2, config2)
print(f"AI: {result1['messages'][-1].content}")
print(f"摘要: {result1.get('summary', '无')}")

print("\n第2轮对话:")
user_input2 = "我专门研究机器学习算法"
print(f"用户: {user_input2}")
result2 = graph_summary.invoke({"messages": [HumanMessage(content=user_input2)], "summary": result1.get('summary', '')}, config2)
print(f"AI: {result2['messages'][-1].content}")
print(f"摘要: {result2.get('summary', '无')}")

# 模拟长对话触发摘要
print("\n模拟长对话（添加多条消息）:")
long_messages = []
for i in range(12):  # 超过10条消息触发摘要
    long_messages.append(HumanMessage(content=f"这是第{i+1}条测试消息"))
    long_messages.append(AIMessage(content=f"收到第{i+1}条消息"))

state_long = {"messages": long_messages, "summary": ""}
result_long = graph_summary.invoke(state_long, config2)
print(f"长对话后的摘要: {result_long.get('summary', '无')}")
print(f"保留的消息数量: {len(result_long['messages'])}")

print("\n✅ 方式2测试完成")


🧪 开始测试方式2：带摘要的对话记忆

第1轮对话:
用户: 你好，我叫李四，是一名数据科学家
AI: 你好，李四！很高兴遇到一位数据科学领域的专家。作为数据科学家，您肯定在处理和分析大量数据方面有着丰富的经验和技术知识吧？有什么具体的问题或讨论主题想要探讨吗？无论是技术问题、行业趋势还是项目合作的机会，我都很愿意帮助解答或交流。
摘要: 

第2轮对话:
用户: 我专门研究机器学习算法
AI: 很高兴听到您在研究机器学习算法方面有着专长！如果您有任何关于特定机器学习算法的问题、需要建议参考资料，或者想要讨论某个具体的应用案例和技术细节，请随时告诉我。我可以提供帮助或分享我的知识来加深您的理解。无论是理论探讨还是实践应用，我都乐意参与其中。
摘要: 

模拟长对话（添加多条消息）:
长对话后的摘要: 这段对话中，一方不断发送不同编号的测试消息给另一方。另一方每次收到消息后都会回复“收到XX号消息”。最终，该方成功接收了从第1条到第10条的所有测试消息。
保留的消息数量: 1

✅ 方式2测试完成


## 方式3: 多层记忆系统

In [7]:

class MultiLayerMemoryState(TypedDict):
    messages: Annotated[List[BaseMessage], "当前对话"]
    short_term: Annotated[List[BaseMessage], "短期记忆"]
    long_term: Annotated[List[str], "长期记忆摘要"]
    user_profile: Annotated[Dict[str, Any], "用户档案"]


def update_memory_layers(state: MultiLayerMemoryState) -> dict:
    """更新多层记忆"""
    messages = state["messages"]
    short_term = state.get("short_term", [])
    long_term = state.get("long_term", [])
    user_profile = state.get("user_profile", {})

    # 更新短期记忆
    short_term.extend(messages)
    if len(short_term) > 20:  # 短期记忆超限时转移到长期
        old_messages = short_term[:10]
        summary_text = "\n".join([f"{msg.type}: {msg.content}" for msg in old_messages])
        summary_prompt = f"摘要这段对话：\n{summary_text}"
        summary = model.invoke([HumanMessage(content=summary_prompt)]).content
        long_term.append(summary)
        short_term = short_term[10:]

    # 更新用户档案
    last_message = messages[-1].content if messages else ""
    if any(keyword in last_message.lower() for keyword in ["我叫", "我是", "我的名字", "我住在"]):
        profile_prompt = f"从这句话提取用户信息（JSON格式）：{last_message}"
        try:
            profile_response = model.invoke([HumanMessage(content=profile_prompt)]).content
            # 简化处理，实际应用中需要更好的JSON解析
            if "姓名" in last_message or "名字" in last_message:
                user_profile["name"] = last_message
        except:
            pass

    return {
        "short_term": short_term,
        "long_term": long_term[-5:],  # 保留最近5个长期记忆
        "user_profile": user_profile
    }


def call_model_multilayer(state: MultiLayerMemoryState) -> dict:
    """多层记忆模型调用"""
    system_parts = ["你是一个AI助手。"]

    if state.get("user_profile"):
        system_parts.append(f"用户信息：{state['user_profile']}")

    if state.get("long_term"):
        system_parts.append(f"历史摘要：{'; '.join(state['long_term'])}")

    system_msg = SystemMessage(content="\n".join(system_parts))
    recent_messages = state.get("short_term", [])[-6:]  # 最近3轮对话
    messages = [system_msg] + recent_messages + state["messages"]

    response = model.invoke(messages)
    return {"messages": [response]}


# 创建多层记忆图
builder_multilayer = StateGraph(MultiLayerMemoryState)
builder_multilayer.add_node("update_memory", update_memory_layers)
builder_multilayer.add_node("call_model", call_model_multilayer)
builder_multilayer.add_edge(START, "update_memory")
builder_multilayer.add_edge("update_memory", "call_model")
builder_multilayer.add_edge("call_model", END)

graph_multilayer = builder_multilayer.compile(checkpointer=MemorySaver())
print("✅ 方式3: 多层记忆图创建完成")

✅ 方式3: 多层记忆图创建完成


In [20]:
# 方式3测试：多层记忆系统
print("\n🧪 开始测试方式3：多层记忆系统")
print("="*60)

config3 = {"configurable": {"thread_id": str(uuid.uuid4())}}

print("\n第1轮对话:")
user_input1 = "你好，我叫王五，是一名医生"
print(f"用户: {user_input1}")
state3 = {
    "messages": [HumanMessage(content=user_input1)],
    "short_term": [],
    "long_term": [],
    "user_profile": {}
}
result1 = graph_multilayer.invoke(state3, config3)
print(f"AI: {result1['messages'][-1].content}")
print(f"用户档案: {result1.get('user_profile', {})}")

print("\n第2轮对话:")
user_input2 = "我在北京协和医院工作"
print(f"用户: {user_input2}")
result2 = graph_multilayer.invoke({"messages": [HumanMessage(content=user_input2)], **result1}, config3)
print(f"AI: {result2['messages'][-1].content}")
print(f"短期记忆数量: {len(result2.get('short_term', []))}")

print("\n第3轮对话:")
user_input3 = "你知道我的职业和工作地点吗？"
print(f"用户: {user_input3}")
result3 = graph_multilayer.invoke({"messages": [HumanMessage(content=user_input3)], **result2}, config3)
print(f"AI: {result3['messages'][-1].content}")

# 模拟大量对话触发长期记忆
print("\n模拟大量对话触发长期记忆:")
many_messages = []
for i in range(25):  # 超过20条触发长期记忆
    many_messages.append(HumanMessage(content=f"对话{i+1}"))

state_many = {
    "messages": [HumanMessage(content="最新消息")],
    "short_term": many_messages,
    "long_term": [],
    "user_profile": {"name": "王五", "job": "医生"}
}
result_many = graph_multilayer.invoke(state_many, config3)
print(f"长期记忆数量: {len(result_many.get('long_term', []))}")
print(f"短期记忆数量: {len(result_many.get('short_term', []))}")

print("\n✅ 方式3测试完成")


🧪 开始测试方式3：多层记忆系统

第1轮对话:
用户: 你好，我叫王五，是一名医生
AI: 您好，王五医生！很高兴为您服务。如果在与您的患者交流时遇到任何问题或需要建议，请随时告诉我。您有什么具体的问题或者话题想要讨论吗？我可以提供一些帮助和信息。
用户档案: {}

第2轮对话:
用户: 我在北京协和医院工作
AI: 
短期记忆数量: 2

第3轮对话:
用户: 你知道我的职业和工作地点吗？
AI: 另外，如果您想了解最新的医疗研究成果、医学知识更新或是关于特定疾病的治疗方案等，也可以问我，我会尽力为您提供相关信息。请随时告知！

模拟大量对话触发长期记忆:
长期记忆数量: 1
短期记忆数量: 16

✅ 方式3测试完成


## 方式4: 跨线程持久化记忆 (使用Store)

In [8]:
def call_model_with_store(state: MessagesState, config: RunnableConfig, *, store) -> dict:
    """使用Store的跨线程记忆"""
    user_id = config["configurable"].get("user_id", "default")
    namespace = ("memories", user_id)

    # 搜索相关记忆
    last_message = state["messages"][-1]
    memories = store.search(namespace, query=str(last_message.content))
    memory_info = "\n".join([d.value.get("data", "") for d in memories])

    # 构建系统消息
    system_content = "你是一个AI助手。"
    if memory_info:
        system_content += f"\n相关记忆：{memory_info}"

    # 存储新记忆
    if "记住" in last_message.content or "我叫" in last_message.content:
        memory_data = {"data": last_message.content, "timestamp": datetime.now().isoformat()}
        store.put(namespace, str(uuid.uuid4()), memory_data)

    system_msg = SystemMessage(content=system_content)
    messages = [system_msg] + state["messages"]
    response = model.invoke(messages)

    return {"messages": [response]}


# 创建Store图
store = InMemoryStore()
builder_store = StateGraph(MessagesState)
builder_store.add_node("call_model", call_model_with_store)
builder_store.add_edge(START, "call_model")
builder_store.add_edge("call_model", END)

graph_store = builder_store.compile(checkpointer=MemorySaver(), store=store)
print("✅ 方式4: Store记忆图创建完成")

✅ 方式4: Store记忆图创建完成


In [21]:
# 方式4测试：跨线程持久化记忆
print("\n🧪 开始测试方式4：跨线程持久化记忆")
print("="*60)

config4 = {"configurable": {"thread_id": str(uuid.uuid4()), "user_id": "test_user_001"}}

print("\n第1轮对话（存储记忆）:")
user_input1 = "记住我叫赵六，我是一名教师"
print(f"用户: {user_input1}")
result1 = graph_store.invoke({"messages": [HumanMessage(content=user_input1)]}, config4)
print(f"AI: {result1['messages'][-1].content}")

print("\n第2轮对话:")
user_input2 = "记住我教数学，在清华大学工作"
print(f"用户: {user_input2}")
result2 = graph_store.invoke({"messages": [HumanMessage(content=user_input2)]}, config4)
print(f"AI: {result2['messages'][-1].content}")

print("\n第3轮对话（测试记忆检索）:")
user_input3 = "我的职业是什么？"
print(f"用户: {user_input3}")
result3 = graph_store.invoke({"messages": [HumanMessage(content=user_input3)]}, config4)
print(f"AI: {result3['messages'][-1].content}")

# 测试跨线程记忆
print("\n测试跨线程记忆（新线程，同用户）:")
config4_new = {"configurable": {"thread_id": str(uuid.uuid4()), "user_id": "test_user_001"}}
user_input4 = "你还记得我的名字吗？"
print(f"用户: {user_input4}")
result4 = graph_store.invoke({"messages": [HumanMessage(content=user_input4)]}, config4_new)
print(f"AI: {result4['messages'][-1].content}")

print("\n✅ 方式4测试完成")


🧪 开始测试方式4：跨线程持久化记忆

第1轮对话（存储记忆）:
用户: 记住我叫赵六，我是一名教师
AI: 好的，我会记住你的名字是赵六，并且你现在是一名教师。如果有任何关于教学或者与教师相关的问题，我会尽力帮助你。你可以随时向我提问或讨论教育相关的主题。

第2轮对话:
用户: 记住我教数学，在清华大学工作
AI: 好的，我已经记住了这些信息：你是赵六，目前在清华大学担任数学教师的职务。如果你有任何关于数学教学、学术研究或者与教学相关的问题，我会尽力帮助你。随时可以向我提问。

第3轮对话（测试记忆检索）:
用户: 我的职业是什么？
AI: 根据你之前提供的信息，你的职业是教师，并且你现在是一名数学教师，在清华大学工作。所以，可以说你的职业是在清华大学担任数学教师。如果你有其他问题或需要进一步的帮助，请告诉我！

测试跨线程记忆（新线程，同用户）:
用户: 你还记得我的名字吗？
AI: 您还没有提到您的名字，所以我并不知道您的名字是何许人也。不过如果您愿意告诉我，我会很高兴地记住您。

✅ 方式4测试完成


## 方式5: 智能路由记忆系统

In [9]:
## 方式5: 智能路由记忆系统
class RoutedMemoryState(TypedDict):
    messages: Annotated[List[BaseMessage], "对话消息"]
    memory_type: Annotated[str, "记忆类型"]
    context: Annotated[Dict[str, Any], "上下文信息"]


def classify_message(state: RoutedMemoryState) -> dict:
    """分类消息类型"""
    last_message = state["messages"][-1].content.lower()

    if any(word in last_message for word in ["记住", "我叫", "我是", "我的"]):
        memory_type = "personal"
    elif any(word in last_message for word in ["什么", "怎么", "为什么", "如何"]):
        memory_type = "question"
    else:
        memory_type = "casual"

    return {"memory_type": memory_type}


def handle_personal_memory(state: RoutedMemoryState) -> dict:
    """处理个人信息记忆"""
    context = state.get("context", {})
    last_message = state["messages"][-1].content

    # 提取个人信息
    if "我叫" in last_message or "我是" in last_message:
        context["personal_info"] = last_message

    system_msg = SystemMessage(content=f"你是AI助手。用户信息：{context.get('personal_info', '无')}")
    messages = [system_msg] + state["messages"]
    response = model.invoke(messages)

    return {"messages": [response], "context": context}


def handle_question_memory(state: RoutedMemoryState) -> dict:
    """处理问题记忆"""
    context = state.get("context", {})

    system_content = "你是AI助手，擅长回答问题。"
    if context.get("personal_info"):
        system_content += f"\n用户信息：{context['personal_info']}"

    system_msg = SystemMessage(content=system_content)
    messages = [system_msg] + state["messages"]
    response = model.invoke(messages)

    return {"messages": [response]}


def handle_casual_memory(state: RoutedMemoryState) -> dict:
    """处理日常对话记忆"""
    system_msg = SystemMessage(content="你是一个友好的AI助手。")
    messages = [system_msg] + state["messages"][-6:]  # 只保留最近对话
    response = model.invoke(messages)

    return {"messages": [response]}


def route_memory_type(state: RoutedMemoryState) -> str:
    """路由到不同的记忆处理节点"""
    return state["memory_type"]


# 创建路由图
builder_routed = StateGraph(RoutedMemoryState)
builder_routed.add_node("classify", classify_message)
builder_routed.add_node("personal", handle_personal_memory)
builder_routed.add_node("question", handle_question_memory)
builder_routed.add_node("casual", handle_casual_memory)

builder_routed.add_edge(START, "classify")
builder_routed.add_conditional_edges(
    "classify",
    route_memory_type,
    {
        "personal": "personal",
        "question": "question",
        "casual": "casual"
    }
)
builder_routed.add_edge("personal", END)
builder_routed.add_edge("question", END)
builder_routed.add_edge("casual", END)

graph_routed = builder_routed.compile(checkpointer=MemorySaver())
print("✅ 方式5: 路由记忆图创建完成")

✅ 方式5: 路由记忆图创建完成


In [22]:
# 方式5测试：智能路由记忆系统
print("\n🧪 开始测试方式5：智能路由记忆系统")
print("="*60)

config5 = {"configurable": {"thread_id": str(uuid.uuid4())}}

print("\n第1轮对话（个人信息类型）:")
user_input1 = "你好，我叫孙七，我是一名工程师"
print(f"用户: {user_input1}")
state5 = {
    "messages": [HumanMessage(content=user_input1)],
    "memory_type": "",
    "context": {}
}
result1 = graph_routed.invoke(state5, config5)
print(f"AI: {result1['messages'][-1].content}")
print(f"分类类型: {result1.get('memory_type', '未知')}")
print(f"上下文: {result1.get('context', {})}")

print("\n第2轮对话（问题类型）:")
user_input2 = "什么是人工智能？"
print(f"用户: {user_input2}")
state5_2 = {
    "messages": [HumanMessage(content=user_input2)],
    "memory_type": "",
    "context": result1.get('context', {})
}
result2 = graph_routed.invoke(state5_2, config5)
print(f"AI: {result2['messages'][-1].content}")
print(f"分类类型: {result2.get('memory_type', '未知')}")

print("\n第3轮对话（日常对话类型）:")
user_input3 = "今天天气不错"
print(f"用户: {user_input3}")
state5_3 = {
    "messages": [HumanMessage(content=user_input3)],
    "memory_type": "",
    "context": result1.get('context', {})
}
result3 = graph_routed.invoke(state5_3, config5)
print(f"AI: {result3['messages'][-1].content}")
print(f"分类类型: {result3.get('memory_type', '未知')}")

print("\n第4轮对话（测试个人信息记忆）:")
user_input4 = "你还记得我的名字和职业吗？"
print(f"用户: {user_input4}")
state5_4 = {
    "messages": [HumanMessage(content=user_input4)],
    "memory_type": "",
    "context": result1.get('context', {})
}
result4 = graph_routed.invoke(state5_4, config5)
print(f"AI: {result4['messages'][-1].content}")
print(f"分类类型: {result4.get('memory_type', '未知')}")

print("\n✅ 方式5测试完成")


🧪 开始测试方式5：智能路由记忆系统

第1轮对话（个人信息类型）:
用户: 你好，我叫孙七，我是一名工程师
AI: 你好！很高兴认识你，孙七。作为一名工程师，你觉得你的工作给社会带来了哪些积极的影响呢？或者有什么特别的挑战吗？
分类类型: personal
上下文: {'personal_info': '你好，我叫孙七，我是一名工程师'}

第2轮对话（问题类型）:
用户: 什么是人工智能？
AI: 人工智能（Artificial Intelligence，简称AI）是指利用计算机模拟、延伸和扩展人的智能的技术。它使机器能够在没有人类明确编程的情况下执行智能任务，涵盖了许多领域，包括学习、推理、规划、问题求解、感知以及语言理解等。

简单来说，就是通过算法和技术让计算机能够模仿或实现人类的智能行为，比如识别图像、处理语音、玩游戏甚至创作艺术作品。随着技术的进步，人工智能已经在很多行业和日常生活中扮演着越来越重要的角色。
分类类型: question

第3轮对话（日常对话类型）:
用户: 今天天气不错
AI: 是的，不错的天气！您打算去做些什么呢？有时候好天气会让人感到更加愉悦和有活力。
分类类型: casual

第4轮对话（测试个人信息记忆）:
用户: 你还记得我的名字和职业吗？
AI: 当然记得，你的名字是孙七，你现在是一名工程师。
分类类型: personal

✅ 方式5测试完成


In [23]:
# 方式6测试：流式对话记忆
print("\n🧪 开始测试方式6：流式对话记忆")
print("="*60)

config6 = {"configurable": {"thread_id": str(uuid.uuid4())}}

print("\n第1轮流式对话:")
user_input1 = "你好，我是周八，请用流式方式回复我"
print(f"用户: {user_input1}")
print("AI: ", end="", flush=True)

full_response = ""
for chunk in graph_basic.stream(
    {"messages": [HumanMessage(content=user_input1)]},
    config6
):
    if "call_model" in chunk and "messages" in chunk["call_model"]:
        content = chunk["call_model"]["messages"][-1].content
        print(content, end="", flush=True)
        full_response += content

print(f"\n完整回复: {full_response}")

print("\n第2轮流式对话:")
user_input2 = "你还记得我刚才说的名字吗？"
print(f"用户: {user_input2}")
print("AI: ", end="", flush=True)

full_response2 = ""
for chunk in graph_basic.stream(
    {"messages": [HumanMessage(content=user_input2)]},
    config6
):
    if "call_model" in chunk and "messages" in chunk["call_model"]:
        content = chunk["call_model"]["messages"][-1].content
        print(content, end="", flush=True)
        full_response2 += content

print(f"\n完整回复: {full_response2}")

print("\n✅ 方式6测试完成")


🧪 开始测试方式6：流式对话记忆

第1轮流式对话:
用户: 你好，我是周八，请用流式方式回复我
AI: 你好周八，很高兴为你服务。请问我可以帮你做什么？
完整回复: 你好周八，很高兴为你服务。请问我可以帮你做什么？

第2轮流式对话:
用户: 你还记得我刚才说的名字吗？
AI: 当然，你刚刚说你的名字是周八。有什么我可以帮助你的吗？
完整回复: 当然，你刚刚说你的名字是周八。有什么我可以帮助你的吗？

✅ 方式6测试完成



## 总结
本notebook展示了6种LangGraph对话记忆实现方法：

1. ** 基础记忆 ** - 使用MessagesState自动管理对话历史
2. ** 摘要记忆 ** - 自动摘要长对话，节省内存
3. ** 多层记忆 ** - 短期 / 长期记忆分层，用户档案管理
4. ** Store记忆 ** - 跨线程持久化，支持搜索
5. ** 路由记忆 ** - 智能分类不同类型的对话
6. ** 流式记忆 ** - 支持实时流式输出

选择建议：
- 简单应用：方式1
- 长对话：方式2
- 复杂场景：方式3、5
- 多用户：方式4
- 实时交互：方式6

LangGraph的优势：
- 自动状态管理
- 内置检查点机制
- 灵活的图结构
- 强大的条件路由

In [3]:

# 定义调用模型的节点函数
def call_model(state: MessagesState) -> dict:
    """调用模型并返回响应"""
    # 添加系统提示
    system_msg = {
        "role": "system",
        "content": "你是一个友好的AI助手，能够记住对话历史并提供有用的回答。"
    }

    # 构建消息列表
    messages = [system_msg] + state["messages"]

    # 调用模型
    response = model.invoke(messages)

    # 返回新消息
    return {"messages": [response]}


# 创建状态图
builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
builder.add_edge("call_model", END)

# 编译图并添加内存检查点
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

print("✅ 基础对话图创建完成")

# 测试基础对话记忆
print("=" * 50)
print("测试基础对话记忆")
print("=" * 50)

# 创建会话配置
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

# 第一轮对话
print("\n第1轮对话:")
user_input1 = "你好，我叫张三，是一名程序员"
print(f"用户: {user_input1}")

result1 = graph.invoke(
    {"messages": [HumanMessage(content=user_input1)]},
    config
)
print(f"AI: {result1['messages'][-1].content}")

# 第二轮对话 - 测试记忆功能
print("\n第2轮对话:")
user_input2 = "你还记得我的名字和职业吗？"
print(f"用户: {user_input2}")

result2 = graph.invoke(
    {"messages": [HumanMessage(content=user_input2)]},
    config
)
print(f"AI: {result2['messages'][-1].content}")

# 第三轮对话
print("\n第3轮对话:")
user_input3 = "我今年30岁，住在北京"
print(f"用户: {user_input3}")

result3 = graph.invoke(
    {"messages": [HumanMessage(content=user_input3)]},
    config
)
print(f"AI: {result3['messages'][-1].content}")

# 第四轮对话 - 测试完整记忆
print("\n第4轮对话:")
user_input4 = "请总结一下我的个人信息"
print(f"用户: {user_input4}")

result4 = graph.invoke(
    {"messages": [HumanMessage(content=user_input4)]},
    config
)
print(f"AI: {result4['messages'][-1].content}")


✅ 基础对话图创建完成
测试基础对话记忆

第1轮对话:
用户: 你好，我叫张三，是一名程序员
AI: 你好，张三！很高兴认识你。作为一名程序员，你觉得编程中最让你感到兴奋的是什么？

第2轮对话:
用户: 你还记得我的名字和职业吗？
AI: 当然记得。你叫张三，是一名程序员。我们之前聊过你在编程方面的兴趣点或是遇到的挑战。有什么我可以帮助你的吗？

第3轮对话:
用户: 我今年30岁，住在北京
AI: 你好，张三！很高兴知道你还这么年轻就从事编程工作了。北京是一个充满活力和技术创新的城市，你在那里生活和工作感觉如何呢？

除此之外，还有其他方面是你想聊的吗？或者你是否需要一些关于编程、技术发展或职业发展的建议和信息？

第4轮对话:
用户: 请总结一下我的个人信息
AI: 好的，张三。根据我们之前的对话，你可以这样简要地总结你的个人信息：

- **姓名**：张三  
- **年龄**：30岁  
- **居住城市**：北京  
- **职业**：程序员  

如果你有任何具体的问题或需要进一步讨论的方面，请告诉我！
